In [ ]:
from subprocess import call
import time
import glob
import os
import sys
import tifffile
from skimage import io
import numpy as np
import nibabel as nib
import multiprocessing as mp
from scipy import signal
import gc
import time

In [ ]:
def is_file_empty(file_path):
    """ Check if file is empty by confirming if its size is 0 bytes"""
    # Check if file exist and it is empty
    return os.path.exists(file_path) and os.stat(file_path).st_size == 0
def Register_single_image(Mov_name,template_name):        
    output_name = Mov_name.replace('.tif','_LongReg')      
    if not os.path.exists(output_name+'.nii') and is_file_empty(output_name+'.nii'):
        #job_string = "antsRegistration -d 3 --float 1 -o [OutImg, OutImg.nii] -n WelchWindowedSinc -w [0.005,0.995] -u 1 -r [FixImg,MovImg, 1] -t rigid[0.1] -m MI[FixImg,MovImg,1,32, Regular,0.25] -c [1000x500x200x50,1e-7,5] -f 8x4x2x1 -s 2x1x1x0vox -t Affine[0.1] -m MI[FixImg,MovImg,1,32, Regular,0.25] -c [1000x500x200x50,1e-7,5] -f 8x4x2x1 -s 2x1x1x0vox -t SyN[0.05,6,0.5] -m CC[FixImg,MovImg,1,2] -c [100x500x200x50,1e-7,5] -f 8x4x2x1 -s 2x2x1x0vox -v 0"            
        job_string = "antsRegistration -d 2 --float 1 -o [OutImg,OutImg.nii] -n WelchWindowedSinc -w [0.05,0.95]  -u 0 - r [FixImg,MovImg] -t Rigid[0.1] -m MI[FixImg,MovImg,1,32,Regular,0.25] -c 500x200x100x50 -f 8x4x2x1 -s 3x2x1x0 -t Affine[0.1] -m MI[FixImg,MovImg,1,32,Regular,0.25] -c 500x200x100x50 -f 8x4x2x1 -s 3x2x1x0 -t SyN[0.1] -m CC[FixImg,MovImg,1,2] -c 100x50x20x10 -f 8x4x2x1 -s 3x2x1x0 -v 1"        
        job_string = job_string.replace('OutImg',output_name).replace('FixImg',template_name).replace('MovImg',Mov_name)
        call([job_string],shell=True)  
def Register_single_image_forced(Mov_name,template_name):        
    output_name = Mov_name.replace('.tif','_LongReg') 
    job_string = "antsRegistration -d 2 --float 1 -o [OutImg,OutImg.nii] -n WelchWindowedSinc -w [0.05,0.95]  -u 0 - r [FixImg,MovImg] -t Rigid[0.1] -m MI[FixImg,MovImg,1,32,Regular,0.25] -c 500x200x100x50 -f 8x4x2x1 -s 3x2x1x0 -t Affine[0.1] -m MI[FixImg,MovImg,1,32,Regular,0.25] -c 500x200x100x50 -f 8x4x2x1 -s 3x2x1x0 -t SyN[0.1] -m CC[FixImg,MovImg,1,2] -c 100x50x20x10 -f 8x4x2x1 -s 3x2x1x0 -v 1"        
    job_string = job_string.replace('OutImg',output_name).replace('FixImg',template_name).replace('MovImg',Mov_name)
    call([job_string],shell=True)  

In [ ]:
N = int((mp.cpu_count()/4)-2)
tif_file_folder=('/mnt/e/Gilles/Felicity/F16/')
os.chdir(os.path.dirname(tif_file_folder))

img_seq_list=glob.glob(tif_file_folder+'/C2*.tif')
template_name='/mnt/e/Gilles/Felicity/AVG_C2-MAX_F16_Pulse.tif'
img_seq_list.sort()
print(img_seq_list[0]+' '+img_seq_list[273])

In [ ]:
p=mp.Pool(processes = N)
result=[p.apply_async(Register_single_image_forced, (img_name,template_name)) for img_name in img_seq_list[0:273]]
for ind_result in result:
    ind_result.get()    

In [ ]:
MC_img_list

In [ ]:
MC_img_list=glob.glob(tif_file_folder+'/C2*_LongReg.nii')
C1_name=MC_img_list[0]
base_img=nib.load(C1_name)
base_img=np.squeeze(np.asarray(base_img.get_fdata(),dtype='float32')).transpose()
C2frames=np.zeros((len(MC_img_list),base_img.shape[1],base_img.shape[0]), dtype='float32')
for img_nb,C2_name in enumerate(MC_img_list):    
    img_nb=int(C2_name.split('_F16')[1].split('_LongReg.nii')[0])
    img_temp=nib.load(C2_name)
    img_temp=img_temp.get_fdata()    
    img_temp=np.squeeze(np.asarray(img_temp,dtype='float32')).transpose()
    C2frames[img_nb,:,:]=img_temp
tifffile.imsave(C2_name.replace('.nii','_3D.tif'),C2frames)

In [ ]:
def MakeListAndHyperstack(tif_file_folder):
    tif_list=glob.glob(tif_file_folder+'/*.tif')
    tif_list.sort()
    if not (os.path.exists(tif_file_folder+'/3Dreg/template.tif') and not is_file_empty(tif_file_folder+'/3Dreg/'+tif_file_folder.split('/')[-2]+'_0.tif')):
        temp=[]
        for idx_nb,file in enumerate(tif_list):
            if idx_nb==0:
                temp=io.imread(file,plugin='pil')
            else:
                temp=np.concatenate((temp,io.imread(file,plugin='pil')),axis=0)
            
        file_name=tif_file_folder.split('/')[-1]
        range2=int(file_name.split('range')[-1].split('_')[0])
        step=int(file_name.split('step')[-1].split('_')[0])
        TrueSlices=(range2/step)+1;
        dims=temp.shape
        temp=np.reshape(temp,[dims[0]/TrueSlices,TrueSlices,dims[1],dims[2]])

        #Computing the difference between points from frame to frame should give the point with least changes
        #std_movie=temp[:,0:200].std(axis=(2,3))
        number_of_frames_to_check=50
        std_movie=np.diff(temp[0:number_of_frames_to_check].reshape([number_of_frames_to_check,temp.shape[1],temp.shape[2]*temp.shape[3]]),axis=-1)
        idx_template=np.argmin(std_movie,axis=0)
        #ignores the 0s and compute the mean index of the lowest changes
        idx_template=int(idx_template[idx_template>0].ravel().mean())

        #tmp_idx=np.argmin(signal.detrend(std_movie))
        #mean image of the 3 points surrounding the minimum movement
        template=temp[idx_template-1:idx_template+2].mean(axis=0)
        directory = os.getcwd()
        try:
            new_dir=tif_file_folder+'/3Dreg'
            os.mkdir(new_dir)
        except:
            print('directory exists')
            
        os.chdir(new_dir)
        template_name=new_dir+'/template.tif'
        tifffile.imsave(template_name,template.astype(np.uint16))
        img_seq_list=list()
         
        for img_nb in range(0,temp.shape[0]):
            img_name=new_dir+'/'+tif_file_folder.split('/')[-1]+'_'+str(img_nb)+'.tif'
            tifffile.imsave(img_name,temp[img_nb].astype(np.uint16))
            img_seq_list.append(img_name)
        del temp,std_movie
        gc.collect()
    return(tif_list)

In [ ]:
Fish_files_folder='/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200730'
os.chdir(os.path.dirname(Fish_files_folder))
N = int((mp.cpu_count()/4)-2)
Fish_list=glob.glob(Fish_files_folder+'/*/')
Fish_list.sort()
for i,tif_file_folder in enumerate(Fish_list):    
    print(tif_file_folder)
    img_seq_list=glob.glob(tif_file_folder+'/3Dreg/*.tif')
    template_name=tif_file_folder+'/3Dreg/template.tif'
    p=mp.Pool(processes = N)
    result=[p.apply_async(Register_single_image, (img_name,template_name)) for img_name in img_seq_list]
    for ind_result in result:
        ind_result.get()    

In [ ]:
Fish_files_folder='/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200731'
os.chdir(os.path.dirname(Fish_files_folder))
N = int((mp.cpu_count()/4)-2)
Fish_list=glob.glob(Fish_files_folder+'/*/')
Fish_list.sort()
for i,tif_file_folder in enumerate(Fish_list):    
    print(tif_file_folder)
    img_seq_list=glob.glob(tif_file_folder+'/3Dreg/*.tif')
    template_name=tif_file_folder+'/3Dreg/template.tif'
    p=mp.Pool(processes = N)
    result=[p.apply_async(Register_single_image, (img_name,template_name)) for img_name in img_seq_list]
    for ind_result in result:
        ind_result.get()    

In [ ]:
Fish_files_folder='/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200727'
os.chdir(os.path.dirname(Fish_files_folder))
N = int((mp.cpu_count()/4)-2)
Fish_list=glob.glob(Fish_files_folder+'/*/')
Fish_list.sort()
for i,tif_file_folder in enumerate(Fish_list):    
    try:
        tif_list=MakeListAndHyperstack(tif_file_folder)
        img_seq_list=glob.glob(tif_file_folder+'/3Dreg/*.tif')
        template_name=tif_file_folder+'/3Dreg/template.tif'
        p=mp.Pool(processes = N)
        result=[p.apply_async(Register_single_image, (img_name,template_name)) for img_name in img_seq_list]
        for ind_result in result:
            ind_result.get()
    except:
        print(tif_file_folder + ' failed')

In [ ]:
Fish_files_folder='/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200728'
os.chdir(os.path.dirname(Fish_files_folder))
N = int((mp.cpu_count()/4)-2)
Fish_list=glob.glob(Fish_files_folder+'/*/')
Fish_list.sort()
for i,tif_file_folder in enumerate(Fish_list):    
    try:
        tif_list=MakeListAndHyperstack(tif_file_folder)
        img_seq_list=glob.glob(tif_file_folder+'/3Dreg/*.tif')
        template_name=tif_file_folder+'/3Dreg/template.tif'
        p=mp.Pool(processes = N)
        result=[p.apply_async(Register_single_image, (img_name,template_name)) for img_name in img_seq_list]
        for ind_result in result:
            ind_result.get()
    except:
        print(tif_file_folder + ' failed')

In [ ]:
Fish_files_folder='/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200729'
os.chdir(os.path.dirname(Fish_files_folder))
N = int((mp.cpu_count()/4)-2)
Fish_list=glob.glob(Fish_files_folder+'/*/')
Fish_list.sort()
for i,tif_file_folder in enumerate(Fish_list):    
    try:
        tif_list=MakeListAndHyperstack(tif_file_folder)
        img_seq_list=glob.glob(tif_file_folder+'/3Dreg/*.tif')
        template_name=tif_file_folder+'/3Dreg/template.tif'
        p=mp.Pool(processes = N)
        result=[p.apply_async(Register_single_image, (img_name,template_name)) for img_name in img_seq_list]
        for ind_result in result:
            ind_result.get()
    except:
        print(tif_file_folder + ' failed')

In [ ]:
Fish_files_folder='/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200730'
os.chdir(os.path.dirname(Fish_files_folder))
N = int((mp.cpu_count()/4)-2)
Fish_list=glob.glob(Fish_files_folder+'/*/')
Fish_list.sort()
for i,tif_file_folder in enumerate(Fish_list):    
    try:
        tif_list=MakeListAndHyperstack(tif_file_folder)
        img_seq_list=glob.glob(tif_file_folder+'/3Dreg/*.tif')
        template_name=tif_file_folder+'/3Dreg/template.tif'
        p=mp.Pool(processes = N)
        result=[p.apply_async(Register_single_image, (img_name,template_name)) for img_name in img_seq_list]
        for ind_result in result:
            ind_result.get()
    except:
        print(tif_file_folder + ' failed')

In [ ]:
Fish_files_folder='/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200731'
os.chdir(os.path.dirname(Fish_files_folder))
N = int((mp.cpu_count()/4)-2)
Fish_list=glob.glob(Fish_files_folder+'/*/')
Fish_list.sort()
for i,tif_file_folder in enumerate(Fish_list):    
    try:
        tif_list=MakeListAndHyperstack(tif_file_folder)
        img_seq_list=glob.glob(tif_file_folder+'/3Dreg/*.tif')
        template_name=tif_file_folder+'/3Dreg/template.tif'
        p=mp.Pool(processes = N)
        result=[p.apply_async(Register_single_image, (img_name,template_name)) for img_name in img_seq_list]
        for ind_result in result:
            ind_result.get()
    except:
        print(tif_file_folder + ' failed')

In [ ]:
Fish_files_folder='/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200727'
os.chdir(os.path.dirname(Fish_files_folder))
Fish_list=glob.glob(Fish_files_folder+'/*/')
Fish_list.sort()
tif_file_folder=Fish_list[0]
MC_img_list=glob.glob(tif_file_folder+'/3Dreg/*.nii')
len(MC_img_list)

In [ ]:
sizes=np.zeros(len(MC_img_list))
for i,img_name in enumerate(MC_img_list):
    sizes[i]=os.stat(img_name).st_size
sum(sizes<sizes.max())

In [ ]:
sizes.max()

In [ ]:
np.where(sizes<34560352)[0].astype('int')

In [ ]:
temp=[MC_img_list[j].replace('_LongReg.nii','.tif') for j in np.where(sizes<34560352)[0].astype(int)]
temp

In [ ]:
temp

In [ ]:
template_name=tif_file_folder+'/3Dreg/template.tif'
N = int((mp.cpu_count()/4)-2)
p=mp.Pool(processes = N)
result=[p.apply(Register_single_image_forced, (img_name,template_name)) for img_name in temp]
for ind_result in result:
    ind_result.get()

In [ ]:
for img_name in temp:
    Register_single_image_forced(img_name,template_name)

In [ ]:
name=MC_img_list[0]    
base_img=nib.load(name)
base_img=np.squeeze(np.asarray(base_img.get_fdata(),dtype='float32'))
base_img.shape

In [ ]:
start=time.time()
time.sleep(10)
time.time()-start

In [ ]:
os.path.exists(C1_name)

In [ ]:
frames=np.zeros((len(MC_img_list)*base_img.shape[2],base_img.shape[1],base_img.shape[0]), dtype='uint16')
for img_nb,C1_name in enumerate(MC_img_list):
    print(img_nb)
    img_nb=int(C1_name.split('power60_')[1].split('_Long')[0])
    img_temp=[]
    start=time.time()
    while not img_temp:
        try:
            img_temp=nib.load(C1_name)
        except nib.filebasedimages.ImageFileError:
            if time.time() - start > 3600:
                print("Time out while accessing :" + os.path.basename(C1_name))
                pass
            else:
                time.sleep(30)
        except:
            print("Unexpected error: " + os.path.basename(C1_name))
            raise
    img_temp=img_temp.get_fdata()    
    img_temp=np.squeeze(np.asarray(img_temp,dtype='uint16'))
    frames[base_img.shape[2]*(img_nb):base_img.shape[2]+base_img.shape[2]*(img_nb),:,:]=np.transpose(img_temp,(2,1,0))
tifffile.imsave(name.replace('.nii','_3D.tif'),frames)

In [ ]:
frames.shape

In [ ]:
tif_file_folder='D:/Gilles_HS_ENS/3Dreg'
list_img = glob.glob(tif_file_folder+'/*Reg.nii')    
list_img.sort()
name=list_img[0]    
base_img=nib.load(name)
base_img=np.squeeze(np.asarray(base_img.get_fdata(),dtype='float32'))

In [ ]:
frames=np.zeros((len(list_img)*21,base_img.shape[1],base_img.shape[0]), dtype='uint16')
for img_nb,C1_name in enumerate(list_img):    
    img_temp=nib.load(C1_name)
    img_temp=img_temp.get_fdata()    
    img_temp=np.squeeze(np.asarray(img_temp,dtype='uint16'))
    frames[21*(img_nb):21+21*(img_nb),:,:]=np.transpose(img_temp,(2,1,0))
tifffile.imsave(name.replace('.nii','_3D.tif'),frames)

In [ ]:
tif_file_folder='D:/Gilles_HS_ENS/3Dreg'
list_img = glob.glob(tif_file_folder+'/*.tif')    
list_img.sort()
list_img

In [ ]:
name=list_img[0]    
base_img=tifffile.imread(name)
base_img=np.squeeze(np.asarray(base_img,dtype='uint16'))

In [ ]:
name

In [ ]:
frames=np.zeros((len(list_img)*25,base_img.shape[1],base_img.shape[-1]), dtype='uint16')
for img_nb,C1_name in enumerate(list_img):    
    img_temp=tifffile.imread(C1_name)         
    frames[25*(img_nb):25+25*(img_nb),:,:]=img_temp
tifffile.imsave(name.replace('_0.tif','_3D.tif'),frames)

In [ ]:
img_temp.shape

In [ ]:
frames.shape

In [ ]:
base_img.shape

In [ ]:
def is_file_empty(file_path):
    """ Check if file is empty by confirming if its size is 0 bytes"""
    # Check if file exist and it is empty
    return os.path.exists(file_path) and os.stat(file_path).st_size == 0
def Register_single_image(Mov_name,template_name):        
    output_name = Mov_name.replace('.nii','_LongReg')      
    if not os.path.exists(output_name+'.nii') and is_file_empty(output_name+'.nii'):
        #job_string = "antsRegistration -d 3 --float 1 -o [OutImg, OutImg.nii] -n WelchWindowedSinc -w [0.005,0.995] -u 1 -r [FixImg,MovImg, 1] -t rigid[0.1] -m MI[FixImg,MovImg,1,32, Regular,0.25] -c [1000x500x200x50,1e-7,5] -f 8x4x2x1 -s 2x1x1x0vox -t Affine[0.1] -m MI[FixImg,MovImg,1,32, Regular,0.25] -c [1000x500x200x50,1e-7,5] -f 8x4x2x1 -s 2x1x1x0vox -t SyN[0.05,6,0.5] -m CC[FixImg,MovImg,1,2] -c [100x500x200x50,1e-7,5] -f 8x4x2x1 -s 2x2x1x0vox -v 0"            
        job_string = "antsRegistration -d 3 --float 1 -o [OutImg,OutImg.nii] -n WelchWindowedSinc -w [0.05,0.95]  -u 0 - r [FixImg,MovImg] -t Rigid[0.1] -m MI[FixImg,MovImg,1,32,Regular,0.25] -c 500x200x100x50 -f 8x4x2x1 -s 3x2x1x0 -t Affine[0.1] -m MI[FixImg,MovImg,1,32,Regular,0.25] -c 500x200x100x50 -f 8x4x2x1 -s 3x2x1x0 -t SyN[0.1] -m CC[FixImg,MovImg,1,2] -c 100x50x20x10 -f 8x4x2x1 -s 3x2x1x0 -v 1"        
        job_string = job_string.replace('OutImg',output_name).replace('FixImg',template_name).replace('MovImg',Mov_name)
        call([job_string],shell=True)  
def Register_single_image_forced(Mov_name,template_name):        
    output_name = Mov_name.replace('.nii','_LongReg') 
    job_string = "antsRegistration -d 3 --float 1 -o [OutImg,OutImg.nii] -n WelchWindowedSinc -w [0.05,0.95]  -u 0 - r [FixImg,MovImg] -t Rigid[0.1] -m MI[FixImg,MovImg,1,32,Regular,0.25] -c 500x200x100x50 -f 8x4x2x1 -s 3x2x1x0 -t Affine[0.1] -m MI[FixImg,MovImg,1,32,Regular,0.25] -c 500x200x100x50 -f 8x4x2x1 -s 3x2x1x0 -t SyN[0.1] -m CC[FixImg,MovImg,1,2] -c 100x50x20x10 -f 8x4x2x1 -s 3x2x1x0 -v 1"        
    job_string = job_string.replace('OutImg',output_name).replace('FixImg',template_name).replace('MovImg',Mov_name)
    call([job_string],shell=True)  

In [ ]:
N = int((mp.cpu_count()/4)-2)
tif_file_folder=('/mnt/d/Gilles_GF/20211015_F16_Piece_2_r2/3D_test/')
os.chdir(os.path.dirname(tif_file_folder))

img_seq_list=glob.glob(tif_file_folder+'/C2*.nii')
template_name='/mnt/d/Gilles_GF/20211015_F16_Piece_2_r2/3D_test/template.nii'
img_seq_list.sort()
print(img_seq_list[0])

In [ ]:
p=mp.Pool(processes = N)
result=[p.apply_async(Register_single_image_forced, (img_name,template_name)) for img_name in img_seq_list[0:273]]
for ind_result in result:
    ind_result.get()    

In [ ]:
MC_img_list=glob.glob(tif_file_folder+'/C2*_LongReg.nii')
MC_img_list.sort()
MC_img_list

In [ ]:
MC_img_list=glob.glob(tif_file_folder+'/C2*_LongReg.nii')
C1_name=MC_img_list[0]
base_img=nib.load(C1_name)
base_img=np.squeeze(np.asarray(base_img.get_fdata(),dtype='float32')).transpose()
C2frames=np.zeros((len(MC_img_list),5,base_img.shape[1],base_img.shape[2]), dtype='float32')
for img_nb,C2_name in enumerate(MC_img_list):        
    img_temp=nib.load(C2_name)
    img_temp=img_temp.get_fdata()    
    img_temp=np.squeeze(np.asarray(img_temp,dtype='float32')).transpose()
    C2frames[img_nb,:,:,:]=img_temp    
tifffile.imsave(C2_name.replace('.nii','_3D.tif'),C2frames)


In [ ]:
base_img.shape

In [ ]:
len(MC_img_list)

In [ ]:
def is_file_empty(file_path):
    """ Check if file is empty by confirming if its size is 0 bytes"""
    # Check if file exist and it is empty
    return os.path.exists(file_path) and os.stat(file_path).st_size == 0
def Register_single_image_3D(Mov_name,template_name):        
    output_name = Mov_name.replace('.nii','_Greedy')
    Affine_name = Mov_name.replace('.nii','_Greedy_affine.mat')
    #if not os.path.exists(output_name+'_Greedy_affine.mat') and is_file_empty(output_name+'.nii'):            
    job_string = "greedy -d 3 -a -float -o Affine_name -i FixImg MovImg -n 200x80x40 -ia-identity -V 0"
    job_string = job_string.replace('Affine_name',Affine_name).replace('OutImg',output_name).replace('FixImg',template_name).replace('MovImg',Mov_name)
    call([job_string],shell=True)  
    print(job_string)
    job_string = "greedy -d 3 -float -o OutImg.nii -i FixImg MovImg -it Affine_name -n 200x80x40 -e 1 -V 0"
    job_string = job_string.replace('Affine_name',Affine_name).replace('OutImg',output_name).replace('FixImg',template_name).replace('MovImg',Mov_name)
    call([job_string],shell=True)  
    job_string = "greedy -d 3 -rf FixImg -rm MovImg MovImg_Warped.nii -r OutImg.nii Affine_name"
    job_string = job_string.replace('Affine_name',Affine_name).replace('OutImg',output_name).replace('FixImg',template_name).replace('MovImg',Mov_name)
    call([job_string],shell=True)
    
    
def Morph_C1_3D(Mov_name,template_name):
    output_name = Mov_name.replace('.nii','_Greedy')
    Affine_name = Mov_name.replace('.nii','_Greedy_affine.mat')
    Mov_name2 = Mov_name.replace('C2','C1')
    output_name2 = output_name.replace('C2','C1')
    job_string = "greedy -d 3 -rf FixImg -rm MovImg2 MovImg2_Warped.nii -r OutImg.nii Affine_name"
    job_string = job_string.replace('Affine_name',Affine_name).replace('OutImg',output_name).replace('FixImg',template_name).replace('MovImg2',Mov_name2)
    call([job_string],shell=True)
    
def Register_single_imageb_3D(Mov_name,template_name):        
    output_name = Mov_name.replace('.nii','_Greedy')
    Affine_name = Mov_name.replace('.nii','_Greedy_affine2.mat')
    #if not os.path.exists(output_name+'_Greedy_affine.mat') and is_file_empty(output_name+'.nii'):            
    job_string = "greedy -d 3 -a -float -o Affine_name -i FixImg MovImg -n 100x40x20 -e 0.25 -ia-image-centers -m NCC 3x3x1 -V 0"
    job_string = job_string.replace('Affine_name',Affine_name).replace('OutImg',output_name).replace('FixImg',template_name).replace('MovImg',Mov_name)
    call([job_string],shell=True)  
    print(job_string)
    job_string = "greedy -d 3 -float -o OutImg.nii -i FixImg MovImg -it Affine_name"# -n 100x50x20 -e 0.25"# -m NCC 3x3x1 -V 0"
    job_string = job_string.replace('Affine_name',Affine_name).replace('OutImg',output_name).replace('FixImg',template_name).replace('MovImg',Mov_name)
    call([job_string],shell=True)  
    job_string = "greedy -d 3 -rf FixImg -rm MovImg MovImg_Warped2.nii -r OutImg.nii Affine_name"
    job_string = job_string.replace('Affine_name',Affine_name).replace('OutImg',output_name).replace('FixImg',template_name).replace('MovImg',Mov_name)
    call([job_string],shell=True)
    Mov_name2 = Mov_name.replace('C2','C1')
    output_name2 = output_name.replace('C2','C1')
    job_string = "greedy -d 3 -rf FixImg -rm MovImg2 MovImg2_Warped2.nii -r OutImg.nii Affine_name"
    job_string = job_string.replace('Affine_name',Affine_name).replace('OutImg',output_name).replace('FixImg',template_name).replace('MovImg2',Mov_name2)
    call([job_string],shell=True)

In [ ]:
img_seq_list=glob.glob('/mnt/d/Gilles_GF/20211015_F16_Piece_2_r2/3D_test/C2'+'*.nii')
template_name='/mnt/d/Gilles_GF/20211015_F16_Piece_2_r2/3D_test/template.nii'
for img_name in img_seq_list:
    Register_single_imageb_3D(img_name,template_name)   